In [155]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import datetime
import matplotlib.pyplot as plt

In [156]:
def calc_proportion(array_TF):
    return sum(array_TF)/len(array_TF)


def calc_standard_error(p_f, n_f, phat_f, nhat_f, num_samples_f=1):
    # if we assume one sample (Variant B) and Variant A as baseline = population, p_f and n_f are variant A stats
    if num_samples_f == 1:
        std_err_f = np.sqrt(p_f*(1-p_f)/nhat_f)
        print('Std err for %d sample test: np.sqrt(%3.3f * (1 - %3.3f)/ %d) = %3.4f' % (num_samples_f, p_f, p_f, nhat_f, std_err_f))
        return std_err_f
    # if we assume two samples (Variant A and Variant B both), p_f, n_f, phat_f, nhat_f are the values for each sample
    if num_samples_f == 2:
        std_err_f = np.sqrt((p_f*(1-p_f)/n_f) + (phat_f * (1-phat_f) / nhat_f))
        print('Std err for %d sample test: np.sqrt((%3.3f*(1-%3.3f)/%d) + (%3.3f * (1-%3.3f) / %d))' % (num_samples_f, p_f, p_f, n_f, phat_f, phat_f, nhat_f))
        return std_err_f


def calc_zscore(phat_f, nhat_f, p_f, n_f=1, num_samples_f=1, two_sample_diff_f=0):
    print('Conducting test assuming %d samples' % num_samples_f)
    print('z = ((%3.3f - %3.3f) - %d)/std_error' % (phat_f, p_f, two_sample_diff_f))
    return ((phat_f - p_f) - two_sample_diff_f)/calc_standard_error(p_f, n_f, phat_f, nhat_f, num_samples_f)


def get_z_crit_value(alpha_f, num_sides_f):
    return norm.ppf(1-(alpha_f/num_sides_f))


def get_p_value(zscore_f, num_sides_f):
    return 1 - ((1-norm.cdf(abs(zscore_f))) * num_sides_f)


def reject_null(variantA_outcomes_f, variantB_outcomes_f, alpha_f, num_sides_f, num_samples_f):
    phat_f = calc_proportion(variantB_outcomes_f)
    nhat_f = len(variantB_outcomes_f)
    p_f = calc_proportion(variantA_outcomes_f)
    n_f = len(variantA_outcomes_f)
    print('Proportion 1 (Variant A): %2.3f (%d obs)' % (p_f, n_f))
    print('Proportion 2 (Variant B): %2.3f (%d obs)' % (phat_f, nhat_f))
    z_score_f = calc_zscore(phat_f, nhat_f, p_f, n_f, num_samples_f)
    p_value_f = get_p_value(z_score_f, num_sides_f)
    z_crit_f = get_z_crit_value(alpha_f, num_sides_f)

    return z_score_f > z_crit_f, z_score_f, p_value_f


def calc_optimal_sample_size(p0_f, mde_f, alpha_f, power_f):
    t_alpha2 = abs(norm.ppf(alpha_f / 2))   # t-value corresponding to probability of committing a type 1 error in a two sided test; using z instead of t since sample should be large enough and DOF is unknown
    t_beta = abs(norm.ppf((1 - power_f)/2))     # t-value corresponding to probability of committing a type 2 error in a two sided test; using z instead of t since sample should be large enough and DOF is unknown

    p1_f = p0_f + mde_f     # assumes baseline is provided and lift is in direction provided by MDE
    p_avg = (p0_f + p1_f) / 2

    sample_size = (t_alpha2*np.sqrt(2*p_avg*(1-p_avg)) + t_beta*np.sqrt(p0_f*(1-p0_f) + p1_f*(1-p1_f)))**2 * (1/(mde_f**2))
    return sample_size


def sequential_stopping_test(variantB_outcomes_f, p0_f, mde_f, alpha_f, power_f, boundary_f='Wald'):
    def calc_z(p0_ff, p1_ff):
        return np.log(p1_ff/p0_ff)

    beta_f = 1-power_f
    p1_f = p0_f + mde_f

    if boundary_f == 'Wald':
        # Wald
        A = (1 - beta_f) / alpha_f
        B = beta_f / (1 - alpha_f)
    else:
        # conservative
        A = 1/alpha_f
        B = beta_f

    lower_boundary = np.log(B)
    upper_boundary = np.log(A)

    log_lambda = 0
    n_iter = 0
    log_lambda_list = list()
    for t_outcome in variantB_outcomes_f:
        z = calc_z(p0_ff=abs(p0_f-(1-t_outcome)), p1_ff=abs(p1_f-(1-t_outcome)))
        log_lambda = log_lambda + z
        log_lambda_list.append(log_lambda)
        n_iter = n_iter + 1
        # check stopping criteria
        if log_lambda < lower_boundary:
            return {'reject null': False, 'num_iter': n_iter, 'A': A, 'B': B, 'log_lambda': log_lambda_list}
        elif log_lambda > upper_boundary:
            return {'reject null': True, 'num_iter': n_iter, 'A': A, 'B': B, 'log_lambda': log_lambda_list}

    # if test looks at all data points
    return {'reject null': False, 'num_iter': n_iter, 'A': A, 'B': B, 'log_lambda': log_lambda_list}

## A/B test number

## set trial month

In [157]:
# ----- Provide parameters
trial_start_date = datetime.date(year=2019, month=7, day=1)

## read data

In [158]:


# ----- Read in data -----
df = pd.read_csv('subscribers.csv')


In [159]:
df = df.loc[:,['label','plan_type','account_creation_date','subid']]
df = df.rename({'account_creation_date':'date'},axis = 1)
df = df.rename({'subid':'id'},axis = 1)

## choose variance A and B
## use conversion rate

In [160]:
# choose variance A and B
df = df[df['plan_type'].isin(['base_uae_14_day_trial','low_uae_no_trial'])]


In [161]:
df['plan_type'].unique()

array(['base_uae_14_day_trial', 'low_uae_no_trial'], dtype=object)

In [162]:
df['variant'] = df['plan_type'].apply(lambda x: 'A' if x == 'base_uae_14_day_trial' else 'B' )
df['convert_tf'] = df['label'].apply(lambda x: True if x in ['churn','retain'] else False )

In [163]:
df['plan_type'] = df['variant']
df = df.rename({'plan_type':'Variant'},axis = 1)
df = df.drop('variant', axis = 1)
df['label'] = df['convert_tf']
df = df.rename({'label':'Convert_TF'},axis = 1)
df = df.drop('convert_tf', axis = 1)

In [164]:
df

,Convert_TF,Variant,date,id
0,True,A,1/24/2020 21:44,21724479
1,True,A,3/1/2020 15:44,23383224
2,False,A,12/7/2019 16:37,26844789
3,False,A,1/27/2020 16:09,29417030
4,True,A,10/5/2019 12:57,26723159
...,...,...,...,...
227623,True,A,11/17/2019 14:12,21434712
227624,True,A,12/6/2019 18:02,25843074
227625,True,A,12/21/2019 19:40,24799085
227626,True,A,1/17/2020 23:58,21308040


In [165]:
df.date = pd.to_datetime(df.date, format='%m/%d/%Y %H:%M')    # parse string format

In [166]:
df.date = df.date.apply(lambda x: datetime.date(year=x.year, month=x.month, day=x.day)) # convert to standard (non-pandas) format for comparison against other dates

In [167]:
df

,Convert_TF,Variant,date,id
0,True,A,2020-01-24,21724479
1,True,A,2020-03-01,23383224
2,False,A,2019-12-07,26844789
3,False,A,2020-01-27,29417030
4,True,A,2019-10-05,26723159
...,...,...,...,...
227623,True,A,2019-11-17,21434712
227624,True,A,2019-12-06,25843074
227625,True,A,2019-12-21,24799085
227626,True,A,2020-01-17,21308040


In [168]:
# ----- Get summary stats -----
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

In [170]:
df_summary = df[['year', 'month', 'Variant', 'id', 'Convert_TF']].groupby(['year', 'month', 'Variant']).agg({'id': 'count', 'Convert_TF': 'sum'}).rename(columns={'id': 'num_exposures', 'Convert_TF': 'num_paying'})

In [171]:
df_summary['conv_rate'] = df_summary['num_paying']/df_summary['num_exposures']

In [172]:
perc_vA = df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_paying'] / df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_exposures']
perc_vB = df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_paying'] / df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_exposures']

In [173]:
df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_exposures']

25554.0

In [174]:
print('For month beginning %s, Variant A had %d exposures (%3.1f%%) and Variant B had %d exposures (%3.1f%%)' % (trial_start_date, int(df_summary.loc[(trial_start_date.year, trial_start_date.month, 'A'), 'num_exposures']), perc_vA*100, int(df_summary.loc[(trial_start_date.year, trial_start_date.month, 'B'), 'num_exposures']), perc_vB*100))

For month beginning 2019-07-01, Variant A had 25554 exposures (26.5%) and Variant B had 144 exposures (52.8%)


In [202]:
df['date'][1]

datetime.date(2020, 3, 1)

In [220]:
df_date = df.loc[df['date'] >= datetime.date(2019, 7, 1), :]
df_date = df_date.loc[df_date['date'] <= datetime.date(2019, 7, 31)]

In [221]:
df_date

,Convert_TF,Variant,date,id,year,month
10,True,A,2019-07-11,21191741,2019,7
14,False,A,2019-07-06,26308559,2019,7
19,False,A,2019-07-06,21760199,2019,7
23,False,A,2019-07-13,29576692,2019,7
38,False,A,2019-07-04,25698109,2019,7
...,...,...,...,...,...,...
227586,True,A,2019-07-08,23189569,2019,7
227597,True,A,2019-07-03,28985855,2019,7
227602,False,A,2019-07-21,29909208,2019,7
227606,False,A,2019-07-25,25927672,2019,7


In [181]:
variantA_outcomes

Series([], Name: Convert_TF, dtype: bool)

In [223]:
# ------ Question 1 ------
# set parameters
alpha = 0.05    # significance level
num_sides = 1   # one-sided=1 or two-sided=2 test
num_samples = 1 # treat Variant A as population or sample

# set trial data
df_date = df.loc[df['date'] >= datetime.date(2019, 7, 1), :]
df_date = df_date.loc[df_date['date'] <= datetime.date(2019, 7, 31)]


variantA_outcomes = df_date.loc[df_date['Variant'] == 'A', 'Convert_TF']
variantB_outcomes = df_date.loc[df_date['Variant'] == 'B', 'Convert_TF']

# --- conduct tests
print('\nRun hypothesis test:')
if num_samples == 1:
    reject_null_test, z_score, p_value = reject_null(variantA_outcomes, variantB_outcomes, alpha, num_sides, num_samples)
if num_samples == 2:
    # if Variant A is treated as a sample, use August data only
    reject_null_test, z_score, p_value = reject_null(df.loc[(df['Variant'] == 'A') & (df.date >= trial_start_date), 'purchase_TF'],
                                                     df.loc[(df['Variant'] == 'B') & (df.date >= trial_start_date), 'purchase_TF'],
                                                     alpha, num_sides, num_samples)
print('For %d-sided, %d-sample test, reject null T/F?: %s' % (num_sides, num_samples, reject_null_test))
print('z-score = %3.3f and p-value = %3.1f%%' % (z_score, p_value*100))


Run hypothesis test:
Proportion 1 (Variant A): 0.265 (25554 obs)
Proportion 2 (Variant B): 0.528 (144 obs)
Conducting test assuming 1 samples
z = ((0.528 - 0.265) - 0)/std_error
Std err for 1 sample test: np.sqrt(0.265 * (1 - 0.265)/ 144) = 0.0368
For 1-sided, 1-sample test, reject null T/F?: True
z-score = 7.135 and p-value = 100.0%
